# 基于嵌入和 BM25 混合检索

## 准备

In [1]:
%%time
%%capture

# 安装所需的库

!pip install llama-index-vector-stores-qdrant
!pip install qdrant_client
!pip install trafilatura

!pip install rank_bm25
!pip install nltk jieba

!pip install llama-index-retrievers-bm25==0.1.3
!pip install rank_bm25
!pip install jieba

CPU times: user 58.1 ms, sys: 23.1 ms, total: 81.2 ms
Wall time: 26.7 s


In [2]:
%time

# 加载llm和embeddings

%run ../utils2.py

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model = get_embedding()

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


## 加载文档

In [4]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

documents = TrafilaturaWebReader().load_data(
    ["https://baike.baidu.com/item/颐和园/63458"]
)

len(documents)

CPU times: user 194 ms, sys: 4.27 ms, total: 199 ms
Wall time: 269 ms


1

In [5]:
documents[0].text[:500]

'收藏\n查看我的收藏\n0有用+1\n颐和园，中国清朝时期皇家园林，前身为清漪园，坐落在北京西郊，距城区15千米，全园占地3.009平方千米（其中颐和园世界文化遗产区面积为2.97平方千米），水面约占四分之三。 [18]与圆明园毗邻。它是以昆明湖、万寿山为基址，以杭州西湖为蓝本，汲取江南园林的设计手法而建成的一座大型山水园林，也是保存最完整的一座皇家行宫御苑，被誉为“皇家园林博物馆”。\n清朝乾隆皇帝继位以前，在北京西郊一带，建起了四座大型皇家园林。乾隆十五年（1750年），乾隆皇帝为孝敬其母崇庆皇太后动用448万两白银把这里改建为清漪园，形成了从现清华园到香山长达二十千米的皇家园林区。咸丰十年（1860年），清漪园被英法联军焚毁。光绪十四年（1888年）重建，改称颐和园，作消夏游乐地。光绪二十六年（1900年），颐和园又遭“八国联军”的破坏，珍宝被劫掠一空。清朝灭亡后，颐和园在军阀混战和国民党统治时期，又遭破坏。\n颐和园暑期取消周一闭馆2024-07-14 10:13\n7月14日，颐和园官方发布公告，为满足市民游客暑期游览需求，提供优质文旅服务，即日起至8月31日，取消园中园周一闭园（馆）规'

In [6]:
%%time

from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=128, chunk_overlap=10)
nodes = splitter.get_nodes_from_documents(documents)

len(nodes)

CPU times: user 289 ms, sys: 69 µs, total: 289 ms
Wall time: 288 ms


136

## 向量索引存储

In [7]:
%%time

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("dense_vectors")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(
    docstore=docstore, vector_store=vector_store
)

index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

CPU times: user 1.23 s, sys: 53 ms, total: 1.28 s
Wall time: 14.9 s


## BM25

In [8]:
%%time

# 下载停用词

# 设置 HTTP 代理环境变量
# https://github.com/nltk/nltk_data/issues/154#issuecomment-2144880495
http_proxy="http://192.168.0.134:7890"

import nltk
nltk.set_proxy(f'{http_proxy}')
nltk.download('stopwords')

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 366 ms


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
%%time

import jieba
from typing import List
from nltk.corpus import stopwords

def chinese_tokenizer(text: str) -> List[str]:
    # Use jieba to segment Chinese text
    return list(jieba.cut(text))
    # return list(jieba.lcut(text))

CPU times: user 68.3 ms, sys: 8.05 ms, total: 76.3 ms
Wall time: 76.2 ms


## retriver

In [11]:
%%time

from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever

import nest_asyncio

nest_asyncio.apply()

from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, 
            similarity_top_k=2,
            tokenizer=chinese_tokenizer,
        ),
    ],
    num_queries=1,
    use_async=True,
)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.416 seconds.
Prefix dict has been built successfully.


CPU times: user 418 ms, sys: 44.2 ms, total: 463 ms
Wall time: 457 ms


In [12]:
%%time

nodes = retriever.retrieve("颐和园在哪里?")
print("\n\n".join(node.text for node in nodes))

光绪三十年（1904年），西苑电灯公所恢复发电；同年五月，电灯重新在颐和园亮了起来。
光绪二十四年（1898年）四月二十八日（6月16日），光绪帝在颐和园仁寿殿召见康有为，命康在总署章京上行走，并许其专折奏事。

两天后，她发动政变，囚禁光绪帝，逮捕杀害维新派人士，戊戌变法失败。在变法期间（6月至9月间），慈禧一直住在颐和园。颐和园成为守旧派反对变法、准备政变的中枢 [15]。
CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 117 ms


In [15]:
%%time

nodes = retriever.retrieve("介绍下颐和园文昌院")
print("\n\n".join(node.text for node in nodes))

| |
东宫门 | ||
清晏舫 | ||
文昌院 | 文昌院位于颐和园内文昌阁之东，是中国古典园林中规模最大、品级最高的文物陈列馆。

颐和园联票：60元/张（旺季），50元/张（淡季），有半价票。1、联票包括门票和园中园门票 （园中园包括文昌院、德和园、佛香阁和苏州街）。2、门票和联票均有优惠票，优惠门票：15元。
CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 144 ms


## query

In [13]:
%%time

from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)
response = query_engine.query("颐和园在哪里?")
print(response)

颐和园位于中国北京市，是中国清朝时期的皇家园林。
CPU times: user 93 ms, sys: 8.02 ms, total: 101 ms
Wall time: 3.22 s


In [14]:
%%time

response = query_engine.query("介绍下颐和园文昌院")
print(response)

文昌院位于颐和园内，紧邻文昌阁。作为中国古典园林中规模宏大、等级最高的文物陈列馆，它在颐和园内的地位颇为显著。如果您对历史文物感兴趣或是想要深入了解中国传统文化，文昌院是不可错过的一站。

关于门票信息：
- 正常情况下（旺季），一张颐和园联票的价格为60元人民币。
- 在淡季时，联票价格则会降低至50元人民币。
- 对于学生、老年人等特定群体，提供半价票优惠。

联票包含了门票以及进入园中的一些特别景点的费用。这些景点包括文昌院、德和园、佛香阁和苏州街。如果您单独购买门票，对于某些特定区域可能还有优惠票价，比如15元人民币的优惠门票。
CPU times: user 17.9 ms, sys: 1 µs, total: 17.9 ms
Wall time: 3.86 s
